In [51]:
import openmm
import openmm.app
import openmm.unit
from copy import deepcopy

In [29]:
pdb = openmm.app.PDBFile('gcmc-ghosts.pdb')

In [30]:
ff = openmm.app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml','phenol_openff.xml')

In [31]:
%%timeit
system = ff.createSystem(pdb.topology)

348 ms ± 27.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
system = ff.createSystem(pdb.topology)

In [33]:
%%timeit

integrator = openmm.LangevinIntegrator(300*openmm.unit.kelvin, 1/openmm.unit.picosecond, 0.002*openmm.unit.picoseconds)


13.8 μs ± 277 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [34]:
%%timeit

integrator = openmm.LangevinIntegrator(300*openmm.unit.kelvin, 1/openmm.unit.picosecond, 0.002*openmm.unit.picoseconds)
simulation = openmm.app.Simulation(pdb.topology, system, integrator)

192 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
integrator = openmm.LangevinIntegrator(300*openmm.unit.kelvin, 1/openmm.unit.picosecond, 0.002*openmm.unit.picoseconds)
simulation = openmm.app.Simulation(pdb.topology, system, integrator, openmm.Platform.getPlatformByName("Reference"))

In [61]:
%%timeit
simulation.context.setPositions(pdb.positions)

19.9 ms ± 102 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
simulation.context.setPositions(pdb.positions)

In [63]:
%%timeit

simulation.context.getState()

1.24 μs ± 260 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [64]:
%%timeit

simulation.context.getState(getPositions=True)

66.6 μs ± 299 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [65]:
%%timeit

simulation.context.getState(getEnergy=True)

3.3 s ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%%timeit

simulation.context.getState(getPositions=True, getEnergy=True)

3.41 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit
simulation.context.getState(getPositions=True, getEnergy=True).getPotentialEnergy()

3.31 s ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
simulation.context.getState(getPositions=True)
simulation.context.getState(getEnergy=True)

1.12 ms ± 18.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [43]:
for f in range(system.getNumForces()):
        force = system.getForce(f)
        if force.__class__.__name__ == "NonbondedForce":
            nonbonded_force = force

In [46]:
%%timeit
nonbonded_force.updateParametersInContext(simulation.context)

1.02 ms ± 4.51 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [47]:
%%timeit
nonbonded_force.setParticleParameters(0, 0,0,0)

603 ns ± 2.89 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [49]:
%%timeit
for atom_id in range(14000):
    nonbonded_force.setParticleParameters(0, 0,0,0)

8.83 ms ± 145 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [68]:
%%timeit
positions = deepcopy(pdb.positions)

14.6 ms ± 53.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
